# 分類模型
預計使用: 
>  1. SVM
>  2. KNN
>  3. Naive Bayes

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import heapq
import math

In [2]:
import warnings
warnings.filterwarnings("ignore")

# SVM + k-fold CV
    
**提供3類的機率，用分3類的機率當作權重算accuracy，進去cv選 C**

**kernel 選擇: 先試 rbf -> polynomial   
分多類的方法: 試 one vs one**

#### 參考資料: 
> 1. sklearn manual (有整理過的，大家想看我再傳):    
https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.predict_proba 

> 2. 分多類的機率算法: **Platt Scaling** :
https://axk51013.medium.com/%E6%A8%A1%E5%9E%8B%E5%B0%8D%E8%87%AA%E5%B7%B1%E6%9C%89%E5%A4%9A%E5%B0%91%E4%BF%A1%E5%BF%83-sklearn%E8%A7%A3%E5%AF%86-predict-proba%E6%98%AF%E7%94%9A%E9%BA%BC-part-2-svm-c21f2b856443


In [5]:
from sklearn import svm
from sklearn.svm import SVC

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate

### ** todo ** 
1. 要做幾 fold cross-validation?  
2. C 的選擇 ?? (越大越允許 misclassify，即 soft margin)
3. kernel選擇，我們優先試 rbf -> poly
4. 選參:
    - **rbf**: 
        - gamma : scale 兩點間的influence。gamma 越小，兩點間的距離會被看得越大
    - **polynomial**:
        - degree
        - gamma
        - coef
        
其實我覺得可以試 **rbf** 就好了...，投影到無限維的表現跟投影到有限維 (而且還要自訂...) 比起來，結果不一定比較差我覺得反而會不錯，還可以直接少選兩個參數><
    

### 參考: 各 kernel 效果 、 C & gamma 的選擇

<td><img src="img\kernel_svm.png" alt="Getting started" width=400"/>
<td><img src="img\Cr1.png" alt="Getting started" width=600"/>

In [6]:
# k fold cross validation，k 選多少? 
k= 5  # 分五組   (20% 當 validation 差不多?)

# 參考論文 multisvm 選法...，但會根據資料不同而有不同

# C 選什麼 ?
C = [2**i for i in range(-2, 12+ 1)] # 2的-2次方~ 2的12次方

# gamma 選什麼?
# gamma = [2**i for i in range(-10, 3+1)] # 2的-10次方~ 2的3次方


使用套件預設的 gamma = 'scale'， 1 / (n_features * X.var())，結果較好，<br>
所以這裡只有 C 一個參數要選 ! ! (如果結果不好再試試 polynomial kernel 吧~)

In [7]:
print("要試的 C 的參數有: ", C)

要試的 C 的參數有:  [0.25, 0.5, 1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]


## 提供 3 類的機率

### 分三類的 accuracy 
**原 SVM : 判對 1，判錯 0， accuracy =  sum(判對的1) / total 數量**    
**改 SVM : if 實際的 label 在 SVM 判斷的前 3 名，留下他的機率， if 沒在前 3 名，機率為 0 ，accuracy =  sum(機率) / total 數量**    

**這裡要使用 **改 SVM ** 算出來的 accuracy 來選 C !!!**

### < 分 3 類算 accuracy 的 function >
input: 
- **prob** : 分類模型套件 predict 出的 不同類別的 probabilty matrix (列:每筆資料，欄:不同的label)
- **clf_label** : 要跟什麼 label 比對抓出機率
- **result** : 要不要印出每筆資料前3可能發生的機率，預設是不印 

output: 
- **acc**: 選對的機率當作權重 の accuracy
- **hit_acc**: 3 個中一個就算對 の accuracy

In [14]:
def acc3type(prob, clf_label, result = False):
    
    '''  
    "prob" : sklearn 套件 predict 出的 不同類別的 probabilty matrix (列:每筆資料，欄:不同的label)
    "clf_label" :  用於 與實際label比對抓出機率， ex: 做cv時，是拿 training set 做，所以是trainlabel；做predict時，是拿 testing set 做，所以是testlabel；
    "result" : 要不要印出每筆資料前3可能發生的機率，預設是不印 (做cv時因為只要看accuracy:不印 ； 做predict時想看機率:印)  
    '''
    
    
    p_acc = []
    hit = 0
    
    for l in range(0, len(prob)):

        # 第l筆資料的各種類別機率list
        rlist = prob[l].tolist() 

        # 最大3個機率的位置 (即第幾類)
        p3_loc = heapq.nlargest(5, range(len(rlist)), key= rlist.__getitem__)    # 這語法我找好久... r明明超簡單...

        # 最大的3類 & 對應的機率
        want = [(label[r], round(rlist[r], 3)) for r in p3_loc if rlist[r] != 0]   #給出 (label, p) 的 tuple 且 p = 0 的話我不要
        want.sort(key = lambda x: x[1], reverse= True)  #讓他按照 p 的大小排序
        
        if result:
            print("第", l, "筆資料，最可能發生的 3 種 (label, p): ", want)  # 從第 0 筆開始哦，之後可看看要不要+1 (怕混淆先沒加) 

            
        pa = [p for (i, p) in want if i == clf_label[l]]  # 挑出與實際label相同的機率。實際label有在model判斷的前三名才有機率，沒有的話給空的list
        p_acc = p_acc + pa
        
        if len(pa)!=0:
            hit = hit + 1
            

    acc = sum(p_acc)/len(prob) # 因為實際label可能沒被判成前3名，會給空字串，so要把他當作0算，去除以全部的資料數量，而不是 p_acc 的數量  
    hit_acc = hit/len(prob)
    
    
    if result:
        print("\n")
        print("選對的機率當作權重 の accuracy :" , round(acc, 3))
        print("3 個中一個就算對 の accuracy :" , round(hit_acc, 3))
        
        '''
        利用"acc"，「選對的機率當作權重 の accuracy」來調參數，因為它會影響表情符號出現的機率，也就是會影響推薦順序
        利用"hit_acc"，「3 個中一個就算對 の accuracy」來呈現，因為大家覺得3個有一個是想要的就很棒ㄌ
        '''
        
    
    return acc, hit_acc

### 1. 選定 * 改 svm * 的 參數 C : 使用 Cross Validation 評估，儲存 accuracy 最大的 model

In [32]:
%%time

# 寫進迴圈檢查用什麼 C 做 svm 的 accuracy 會最大
#C_accuracy = []

ac= 0 #預設
hit_ac = 0

for i in C:
    
    # svm model
    # 用預設的 gamma='scale'，少選一個參數，效果也不錯
    # class_weight='balance' : svm 自己針對 imbalance data 的解法，給小群體比較大的 C 值，較不允許misclassify。大家有興趣我們再討論...
    model = svm.SVC(C=i, kernel='rbf', probability=True, class_weight='balanced', 
                    decision_function_shape='ovo', random_state=1)
    
    # k-fold Cross Validation
    kf = KFold(n_splits=k, random_state=1, shuffle=True) # random_state = set.seed()
    prob = cross_val_predict(model, train, trainlabel, cv= kf, n_jobs=-1, method = 'predict_proba')
    
    acc_result = acc3type(prob, trainlabel, result = False)
    accuracy = acc_result[0]
    #C_accuracy.append(round(accuracy, 3))
    
    
    # 若 ac 比較大才儲存 model
    if(accuracy > ac):
        svm_model = model
        ac = accuracy
        hit_ac = acc_result[1]
        best_C = i
        
    print("跑完 C = {}, accuracy = {}".format(i, round(accuracy, 3)))

 
    
print(" ")
print("已儲存 accuracy 最大的 model 為 **svm_model**")
print("choose C = {}, 選對的機率當作權重 の accuracy 為 {}".format(best_C, round(ac, 3)))
print("choose C = {}, 三個中一個就算對 の accuracy 為 {}".format(best_C, round(hit_ac, 3)))
print(" ")

#print("C =", C,"的 accuracy 分別為:", C_accuracy)

跑完 C = 0.25, accuracy = 0.124
跑完 C = 0.5, accuracy = 0.118
跑完 C = 1, accuracy = 0.117
跑完 C = 2, accuracy = 0.118
跑完 C = 4, accuracy = 0.122
跑完 C = 8, accuracy = 0.12
跑完 C = 16, accuracy = 0.119
跑完 C = 32, accuracy = 0.121
跑完 C = 64, accuracy = 0.121
跑完 C = 128, accuracy = 0.121
跑完 C = 256, accuracy = 0.121
跑完 C = 512, accuracy = 0.121
跑完 C = 1024, accuracy = 0.121
跑完 C = 2048, accuracy = 0.121
跑完 C = 4096, accuracy = 0.121
 
已儲存 accuracy 最大的 model 為 **svm_model**
choose C = 0.25, 選對的機率當作權重 の accuracy 為 0.124
choose C = 0.25, 三個中一個就算對 の accuracy 為 0.592
 
Wall time: 1min 38s


### 2. 用決定好的 svm model 做 testing data
注意! 這裡的 testing data 已經知道 label，因為是從training data 偷偷抓出來看的，但實際上是不知道的 !

這是偷吃步偷偷看 accuracy 和檢視分錯的照片!! 實際做的時候看上面 cross validation 的 accuracy 就可以ㄌ，但偷偷看 testing data可以判斷有沒有overfitting

In [106]:
svm_model.fit(train, trainlabel)

# 所有資料被分到每個類別的機率 (matrix)
prob = svm_model.predict_proba(test)  
np.shape(prob)  #有250筆 共5個類別的資料

(250, 5)

In [107]:
acc3type(prob, testlabel, result = True)

第 0 筆資料，最可能發生的 3 種 (label, p):  [(2, 0.832), (3, 0.117), (1, 0.037)]
第 1 筆資料，最可能發生的 3 種 (label, p):  [(1, 0.954), (3, 0.019), (8, 0.012)]
第 2 筆資料，最可能發生的 3 種 (label, p):  [(1, 0.99), (8, 0.006), (2, 0.003)]
第 3 筆資料，最可能發生的 3 種 (label, p):  [(1, 0.98), (2, 0.011), (8, 0.004)]
第 4 筆資料，最可能發生的 3 種 (label, p):  [(1, 0.969), (8, 0.019), (7, 0.006)]
第 5 筆資料，最可能發生的 3 種 (label, p):  [(1, 0.991), (2, 0.003), (8, 0.003)]
第 6 筆資料，最可能發生的 3 種 (label, p):  [(1, 0.984), (7, 0.005), (2, 0.005)]
第 7 筆資料，最可能發生的 3 種 (label, p):  [(1, 0.991), (2, 0.004), (8, 0.003)]
第 8 筆資料，最可能發生的 3 種 (label, p):  [(1, 0.974), (8, 0.013), (2, 0.009)]
第 9 筆資料，最可能發生的 3 種 (label, p):  [(1, 0.947), (2, 0.019), (7, 0.015)]
第 10 筆資料，最可能發生的 3 種 (label, p):  [(1, 0.98), (8, 0.011), (7, 0.004)]
第 11 筆資料，最可能發生的 3 種 (label, p):  [(1, 0.974), (8, 0.014), (2, 0.008)]
第 12 筆資料，最可能發生的 3 種 (label, p):  [(1, 0.989), (2, 0.004), (7, 0.003)]
第 13 筆資料，最可能發生的 3 種 (label, p):  [(1, 0.958), (8, 0.019), (3, 0.012)]
第 14 筆資料，最可能發生的 3 種 (label, p):  

(0.9026240000000001, 1.0)